<a href="https://colab.research.google.com/github/EvaPanou/Applied-Bioinformatics-Thesis/blob/main/metadata_assembly2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# File paths (adjust as needed)
adex_file = '/content/sample_data/Thesis/GSE108497_metadata.tsv'
geo_file = '/content/sample_data/Thesis/GSE108497_series_matrix.txt'
output_file = '/content/sample_data/Thesis/GSE108497_updated_metadata.tsv'

In [4]:
# 1. Load the ADEx metadata (TSV) and preview it.
adex_df = pd.read_csv(adex_file, sep='\t')
adex_df['Sample'] = adex_df['Sample'].astype(str).str.strip().str.strip('"')
print("Preview of ADEx metadata (first and last 5 rows):")
adex_df.head()

Preview of ADEx metadata (first and last 5 rows):


,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity
0,GSM2901826,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino
1,GSM2901827,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino
2,GSM2901828,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino
3,GSM2901829,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino
4,GSM2901830,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,31-40,Not Hispanic or Latino


In [5]:
adex_df.tail()

,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity
507,GSM2902333,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino
508,GSM2902334,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino
509,GSM2902335,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino
510,GSM2902336,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino
511,GSM2902337,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino


In [ ]:
# 2. Extract sample-level characteristics from the GEO file.
sample_ids = None
characteristics_dict = {}

with open(geo_file, 'r') as file:
    for line in file:
        line = line.strip()
        # Get the sample IDs.
        if line.startswith('!Sample_geo_accession'):
            parts = line.split('\t')
            sample_ids = parts[1:]
            # Initialize an entry for each sample.
            for id in sample_ids:
                characteristics_dict[id] = {}
        # Process every line starting with "!Sample_characteristics_ch1"
        elif line.startswith('!Sample_characteristics_ch1'):
            parts = line.split('\t')
            values = parts[1:]
            # If sample_ids are not set or the number of values doesn't match, skip.
            if sample_ids is None or len(values) != len(sample_ids):
                continue
            for i, cell in enumerate(values):
                if cell:
                    # Expect a "property: value" format.
                    if ':' in cell:
                        key, value = cell.split(':', 1)  # Split on the first colon only.
                        key = key.strip()
                        value = value.strip()
                        # Combine duplicate properties by appending with a semicolon.
                        current_cell = characteristics_dict[sample_ids[i]].get(key)
                        if current_cell is None:
                            characteristics_dict[sample_ids[i]][key] = value
                        else:
                            if isinstance(current_cell, list):
                                current_cell.append(value)
                                characteristics_dict[sample_ids[i]][key] = current_cell
                            else:
                                characteristics_dict[sample_ids[i]][key] = [current_cell, value]
                    else:
                        continue

In [6]:
# 2. Extract sample-level characteristics from the GEO file.
sample_ids = None
characteristics_dict = {}

with open(geo_file, 'r') as f:
    for line in f:
        line = line.strip()
        # Extract sample IDs from the GEO file.
        if line.startswith('!Sample_geo_accession'):
            parts = line.split('\t')
            # Clean each sample id by stripping extra spaces and quotes.
            sample_ids = [s.strip().strip('"') for s in parts[1:]]
            for sid in sample_ids:
                characteristics_dict[sid] = {}
        # Process lines starting with '!Sample_characteristics_ch1'
        elif line.startswith('!Sample_characteristics_ch1'):
            parts = line.split('\t')
            values = parts[1:]
            if sample_ids is None:
                continue
            # Adjust the list so its length matches the number of sample IDs.
            if len(values) < len(sample_ids):
                values += [""] * (len(sample_ids) - len(values))
            elif len(values) > len(sample_ids):
                values = values[:len(sample_ids)]
            # Use enumerate to loop over each value with its index.
            for i, cell in enumerate(values):
                if cell:
                    if ':' in cell:
                        key, val = cell.split(':', 1)  # Split on first colon only.
                        # Clean the property name and value.
                        key = key.strip().strip('"')
                        val = val.strip().strip('"')
                        # Check if the property already exists for this sample.
                        current = characteristics_dict[sample_ids[i]].get(key)
                        if current is None:
                            characteristics_dict[sample_ids[i]][key] = val
                        else:
                            # If the property already exists, combine the values.
                            if isinstance(current, list):
                                current.append(val)
                                characteristics_dict[sample_ids[i]][key] = current
                            else:
                                characteristics_dict[sample_ids[i]][key] = [current, val]
                    else:
                        continue


In [7]:
# After processing, convert any lists into a single string (joined by "; ").
for id, props in characteristics_dict.items():
    for key, value in props.items():
        if isinstance(value, list):
            characteristics_dict[id][key] = "; ".join(value)

In [8]:
# 3. Convert the GEO characteristics dictionary into a DataFrame.
geo_characteristics_df = pd.DataFrame.from_dict(characteristics_dict, orient='index')
geo_characteristics_df.index.name = 'Sample'
geo_characteristics_df.reset_index(inplace=True)

In [9]:
# Clean sample IDs in the GEO characteristics DataFrame as well.
geo_characteristics_df['Sample'] = geo_characteristics_df['Sample'].astype(str).str.strip().str.strip('"')
print("Preview of GEO characteristics (first 5 rows):")
geo_characteristics_df.head()

Preview of GEO characteristics (first 5 rows):


,Sample,tissue,grp_p_tp,sample_name,donor_id,age,gender,race,ethnicity,sle,...,fd,nnd,pl_insuff,iugr,sga,batch,time_point,ga_at_collection,ga_at_end_of_pregnancy,if_pe_before_or_after_36_weeks
0,GSM2901826,whole blood,HC_NP_5,HC2013_1,106346,25,Female,C,Not Hispanic or Latino,0,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
1,GSM2901827,whole blood,HC_NP_5,HC2013-15,134642,25,Female,C,Not Hispanic or Latino,0,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
2,GSM2901828,whole blood,HC_NP_5,HC-85,HD-85,25,Female,AA,Not Hispanic or Latino,0,...,0,0,0,0,0,1,NaN,NaN,NaN,NaN
3,GSM2901829,whole blood,HC_NP_5,HC2013-10,139353,24,Female,C,Not Hispanic or Latino,0,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
4,GSM2901830,whole blood,HC_NP_5,HC2013-9,149039,33,Female,C,Not Hispanic or Latino,0,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN


In [10]:
geo_characteristics_df.tail()

,Sample,tissue,grp_p_tp,sample_name,donor_id,age,gender,race,ethnicity,sle,...,fd,nnd,pl_insuff,iugr,sga,batch,time_point,ga_at_collection,ga_at_end_of_pregnancy,if_pe_before_or_after_36_weeks
507,GSM2902333,whole blood,SLE_P_NC_5,231X02H9,T82,37,Female,C,Not Hispanic or Latino,1,...,0,0,0,0,0,4,PP,11.3 WPP,38,NaN
508,GSM2902334,whole blood,SLE_P_NC_5,496-Wnd9,T87,35,Female,AA,Not Hispanic or Latino,1,...,0,0,0,0,0,2,PP,17.4 WPP,40.1,NaN
509,GSM2902335,whole blood,SLE_P_NC_5,U28(136)3MPP,U28,37,Female,H,Not Hispanic or Latino,1,...,0,0,0,0,0,4,PP,13.2 WPP,40.1,NaN
510,GSM2902336,whole blood,SLE_P_NC_5,655X01K9,U68,28,Female,C,Not Hispanic or Latino,1,...,0,0,0,0,0,4,PP,15.3 WPP,37.6,NaN
511,GSM2902337,whole blood,SLE_P_NC_5,723X01K9,U78,30,Female,C,Not Hispanic or Latino,1,...,0,0,0,0,0,4,PP,16.6 WPP,38.2,NaN


In [11]:
# 4. Merge the ADEx metadata with the GEO characteristics DataFrame.
merged_df = adex_df.merge(geo_characteristics_df, on='Sample', how='left')

print("Preview of merged metadata (first 5 rows):")
merged_df.head()

Preview of merged metadata (first 5 rows):


,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity,...,fd,nnd,pl_insuff,iugr,sga,batch,time_point,ga_at_collection,ga_at_end_of_pregnancy,if_pe_before_or_after_36_weeks
0,GSM2901826,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
1,GSM2901827,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
2,GSM2901828,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,1,NaN,NaN,NaN,NaN
3,GSM2901829,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
4,GSM2901830,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN


In [12]:
merged_df.tail()

,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity,...,fd,nnd,pl_insuff,iugr,sga,batch,time_point,ga_at_collection,ga_at_end_of_pregnancy,if_pe_before_or_after_36_weeks
507,GSM2902333,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,0,0,0,0,0,4,PP,11.3 WPP,38,NaN
508,GSM2902334,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,0,0,0,0,0,2,PP,17.4 WPP,40.1,NaN
509,GSM2902335,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,0,0,0,0,0,4,PP,13.2 WPP,40.1,NaN
510,GSM2902336,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,4,PP,15.3 WPP,37.6,NaN
511,GSM2902337,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,4,PP,16.6 WPP,38.2,NaN


In [13]:
# 5. Save the merged DataFrame to a new TSV file.
merged_df.to_csv(output_file, sep='\t', index=False)
print(f"Merged file saved as '{output_file}'")

Merged file saved as '/content/sample_data/Thesis/GSE108497_updated_metadata.tsv'


In [14]:
# 6. (Optional) Reload the saved file and print a preview to verify the output.
reloaded_df = pd.read_csv(output_file, sep='\t')
print("Preview of reloaded merged file (first 5 rows):")
reloaded_df.head()

Preview of reloaded merged file (first 5 rows):


,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity,...,fd,nnd,pl_insuff,iugr,sga,batch,time_point,ga_at_collection,ga_at_end_of_pregnancy,if_pe_before_or_after_36_weeks
0,GSM2901826,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
1,GSM2901827,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
2,GSM2901828,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,1,NaN,NaN,NaN,NaN
3,GSM2901829,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN
4,GSM2901830,GSE108497,Expression,GPL10558,Healthy,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,0,0,0,0,0,3,NaN,NaN,NaN,NaN


In [ ]:
reloaded_df.tail()

,Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity,...,"""fd","""nnd","""pl_insuff","""iugr","""sga","""batch","""time_point","""ga_at_collection","""ga_at_end_of_pregnancy","""if_pe_before_or_after_36_weeks"
507,GSM2902333,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,GSM2902334,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,GSM2902335,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,31-40,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,GSM2902336,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,GSM2902337,GSE108497,Expression,GPL10558,SLE,Whole blood,NaN,Female,21-30,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


also change variable names


remove "" -> CHATGPT

FIND WHY NaN -> chatgpt
